<a href="https://colab.research.google.com/github/AyushDhanai1419/BirdClassifier_Pytorch_Side_Project/blob/master/Train_Notebook_BirdClassifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Developing an AI application

## Bird Classifier 200 different species



# Installing Pytorch

In [0]:
# http://pytorch.org/
from os.path import exists
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'

!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.4.1-{platform}-linux_x86_64.whl torchvision
  
# Installed pillow
!pip install Pillow==4.0.0

# Installed gdown
!pip install gdown==3.6.0


## Importing modules

In [0]:
# Imports here
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import matplotlib.pyplot as plt

import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torchvision import datasets, transforms, models
from torch.optim import lr_scheduler
import seaborn as sns

# imported PIl.image
from PIL import Image


## Downloading Birds Dataset

In [0]:
# Downloadin dataset
my_file_id = "1qaLPSrqFqylRVoh4KrEKs9wpXv7L6sAS"
!gdown https://drive.google.com/uc?id={my_file_id}
!unzip BirdsImages.zip

# Downloading Actual labels
my_file_id = "1v_Vex6cGhrsMF3FAe4WdnhD3iiNxWdJ5"
!gdown https://drive.google.com/uc?id={my_file_id}


## Load the data


In [0]:
from torchvision import datasets
import torchvision.transforms as transforms
from torch.utils.data.sampler import SubsetRandomSampler
import numpy as np

data_dir = 'images'


# TODO: Define transforms for the training data and testing data
train_transforms = transforms.Compose([transforms.RandomRotation(30),
                                       transforms.RandomResizedCrop(224),
                                       transforms.RandomHorizontalFlip(),
                                       transforms.ToTensor(),
                                       transforms.Normalize([0.485, 0.456, 0.406],
                                                            [0.229, 0.224, 0.225])])



# Pass transforms in here, then run the next cell to see how the transforms look
train_data = datasets.ImageFolder(data_dir, transform=train_transforms)



# number of subprocesses to use for data loading
num_workers = 0
# how many samples per batch to load
batch_size = 90
# percentage of training set to use as validation
valid_size = 0.3
again_split = 0.5

# obtain training indices that will be used for validation
num_train = len(train_data)
indices = list(range(num_train))
np.random.shuffle(indices)
split = int(np.floor(valid_size * num_train))
train_idx, valid_idx = indices[split:], indices[:split]


num_train = len(valid_idx)
indicess = list(range(num_train))
split = int(np.floor(again_split * len(valid_idx)))
valid_idx, test_idx = indicess[split:], indicess[:split]

# define samplers for obtaining training and validation batches
train_sampler = SubsetRandomSampler(train_idx)
valid_sampler = SubsetRandomSampler(valid_idx)
test_sampler = SubsetRandomSampler(test_idx)

# prepare data loaders
train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size,
    sampler=train_sampler, num_workers=num_workers)
valid_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, 
    sampler=valid_sampler, num_workers=num_workers)
test_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, 
    sampler=test_sampler, num_workers=num_workers)




In [0]:
# Displaying Total Images

print('Total images : ',len(train_data))
print('Total train images : ',len(train_idx))
print('Total valid images : ',len(valid_idx))
print('Total test images : ',len(test_idx))

Total images :  6033
Total train images :  4224
Total valid images :  905
Total test images :  904


## Label mapping

You'll also need to load in a mapping from category label to category name. You can find this in the file `classes.txt`. 

In [0]:
bird_to_name = {}
with open("/content/classes.txt") as f:
    for line in f:
      key, val = line.rstrip("\n").split('.')
      bird_to_name[int(key)] = val     


In [0]:
print(bird_to_name)

{1: 'Black_footed_Albatross', 2: 'Laysan_Albatross', 3: 'Sooty_Albatross', 4: 'Groove_billed_Ani', 5: 'Crested_Auklet', 6: 'Least_Auklet', 7: 'Parakeet_Auklet', 8: 'Rhinoceros_Auklet', 9: 'Brewer_Blackbird', 10: 'Red_winged_Blackbird', 11: 'Rusty_Blackbird', 12: 'Yellow_headed_Blackbird', 13: 'Bobolink', 14: 'Indigo_Bunting', 15: 'Lazuli_Bunting', 16: 'Painted_Bunting', 17: 'Cardinal', 18: 'Spotted_Catbird', 19: 'Gray_Catbird', 20: 'Yellow_breasted_Chat', 21: 'Eastern_Towhee', 22: 'Chuck_will_Widow', 23: 'Brandt_Cormorant', 24: 'Red_faced_Cormorant', 25: 'Pelagic_Cormorant', 26: 'Bronzed_Cowbird', 27: 'Shiny_Cowbird', 28: 'Brown_Creeper', 29: 'American_Crow', 30: 'Fish_Crow', 31: 'Black_billed_Cuckoo', 32: 'Mangrove_Cuckoo', 33: 'Yellow_billed_Cuckoo', 34: 'Gray_crowned_Rosy_Finch', 35: 'Purple_Finch', 36: 'Northern_Flicker', 37: 'Acadian_Flycatcher', 38: 'Great_Crested_Flycatcher', 39: 'Least_Flycatcher', 40: 'Olive_sided_Flycatcher', 41: 'Scissor_tailed_Flycatcher', 42: 'Vermilion_Fl

# Building and training the classifier


In [0]:
model = models.densenet121(pretrained=True)
model

In [0]:

# Use GPU if it's available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = models.densenet121(pretrained=True)


In [0]:
# TODO: Build and train your network


# Freeze parameters so we don't backprop through them
for param in model.parameters():
    param.requires_grad = False
    
model.classifier = nn.Sequential(nn.Linear(1024, 512),
                                 nn.ReLU(),
                                 nn.Dropout(0.2),
                                 nn.Linear(512, 200),
                                 nn.LogSoftmax(dim=1))

criterion = nn.NLLLoss()

# Only train the classifier parameters, feature parameters are frozen
optimizer = optim.Adam(model.classifier.parameters(), lr=0.001)

# Decay LR by a factor of 0.1 every 8 epochs
scheduler = lr_scheduler.StepLR(optimizer, step_size=4, gamma=0.1)

model.class_to_idx = train_data.class_to_idx
model.to('cuda');


In [0]:
# If Unfreezed Required

for i , param in enumerate(model.parameters()):    #468 parameters
    if(i>300):
      param.requires_grad=True
      
optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=0.001)

In [0]:
model

#Training Model

In [0]:
# number of epochs to train the model
n_epochs = 60

# initialize tracker for minimum validation loss
valid_loss_min = np.Inf # set initial "min" to infinity

for epoch in range(n_epochs):
    # monitor training loss
    train_loss = 0.0
    valid_loss = 0.0
    
    scheduler.step()
    ###################
    # train the model #
    ###################
    model.train() # prep model for training
    for data, target in train_loader:
        
        # Move input and label tensors to the default device
        data, target = data.to(device), target.to(device)
        # clear the gradients of all optimized variables
        optimizer.zero_grad()
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model.forward(data)
        # calculate the loss
        loss = criterion(output, target)
        # backward pass: compute gradient of the loss with respect to model parameters
        loss.backward()
        # perform a single optimization step (parameter update)
        optimizer.step()
        # update running training loss
        train_loss += loss.item()*data.size(0)
        
    ######################    
    # validate the model #
    ######################
    model.eval() # prep model for evaluation
    for data, target in valid_loader:
        # Move input and label tensors to the default device
        data, target = data.to(device), target.to(device)
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model.forward(data)
        # calculate the loss
        loss = criterion(output, target)
        # update running validation loss 
        valid_loss += loss.item()*data.size(0)
        
    # print training/validation statistics 
    # calculate average loss over an epoch
    train_loss = train_loss/len(train_loader.dataset)
    valid_loss = valid_loss/len(valid_loader.dataset)
    
    print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
        epoch+1, 
        train_loss,
        valid_loss
        ))
    
    # save model if validation loss has decreased
    if valid_loss <= valid_loss_min:
        print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
        valid_loss_min,
        valid_loss))
        torch.save(model.state_dict(), 'model.pt')
        valid_loss_min = valid_loss

Epoch: 1 	Training Loss: 3.438211 	Validation Loss: 0.575735
Validation loss decreased (inf --> 0.575735).  Saving model ...
Epoch: 2 	Training Loss: 2.427846 	Validation Loss: 0.392271
Validation loss decreased (0.575735 --> 0.392271).  Saving model ...
Epoch: 3 	Training Loss: 1.793059 	Validation Loss: 0.290282
Validation loss decreased (0.392271 --> 0.290282).  Saving model ...
Epoch: 4 	Training Loss: 1.465282 	Validation Loss: 0.254644
Validation loss decreased (0.290282 --> 0.254644).  Saving model ...
Epoch: 5 	Training Loss: 1.271637 	Validation Loss: 0.213947
Validation loss decreased (0.254644 --> 0.213947).  Saving model ...
Epoch: 6 	Training Loss: 1.107793 	Validation Loss: 0.213665
Validation loss decreased (0.213947 --> 0.213665).  Saving model ...
Epoch: 7 	Training Loss: 0.990791 	Validation Loss: 0.197315
Validation loss decreased (0.213665 --> 0.197315).  Saving model ...
Epoch: 8 	Training Loss: 0.933778 	Validation Loss: 0.207192
Epoch: 9 	Training Loss: 0.854429 

In [0]:
# Loading model.pt 

model.load_state_dict(torch.load('model.pt'))

## Save the checkpoint

Now that your network is trained, save the model so you can load it later for making predictions. You probably want to save other things such as the mapping of classes to indices which you get from one of the image datasets: `image_datasets['train'].class_to_idx`. You can attach this to the model as an attribute which makes inference easier later on.

```model.class_to_idx = image_datasets['train'].class_to_idx```

Remember that you'll want to completely rebuild the model later so you can use it for inference. Make sure to include any information you need in the checkpoint. If you want to load the model and keep training, you'll want to save the number of epochs as well as the optimizer state, `optimizer.state_dict`. You'll likely want to use this trained model in the next part of the project, so best to save it now.

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

In [0]:
model_save_name = 'UnFreezedModel_densenet121_2L_60epochs_batch90.pt'
path = F"/content/gdrive/My Drive/Colab Notebooks/{model_save_name}" 
torch.save(model.state_dict(), path)